**Homework 21**

In this assignment you will create a Convolutional Neural Network to do facial recognition. We begin by importing the dataset of face photos from the previous assignment.

In [198]:
import numpy as np
import matplotlib.pyplot as plt
import torchinfo
from sklearn.model_selection import train_test_split


from sklearn.datasets import fetch_lfw_people
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
faces=lfw_people.images
names=lfw_people.target_names
target=lfw_people.target

We start with an 80/20 train/test split:

In [199]:
faces_train, faces_test, target_train, target_test=train_test_split(faces,target,train_size=0.8)
faces_train.shape
# target_train.shape

(1030, 50, 37)


Here are the imports that you will need:

In [200]:
import torch
import torch.nn as nn
from torch.optim import Adam

In [201]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

The faces dataset is already scaled appropriately, so we can skip that step. Next, we convert the training and testing targets to pytorch tensors and modify their shape.

In [202]:
faces_train=torch.tensor(faces_train).float().reshape(-1,1,50,37)
faces_test=torch.tensor(faces_test).float().reshape(-1,1,50,37)
target_train=torch.tensor(target_train)
target_test=torch.tensor(target_test)

In [203]:
# plt.imshow(faces_train[11],cmap='gray')

In [204]:
faces_train[1].shape

torch.Size([1, 50, 37])

Create a CNN!

In [218]:
# Greyscale so 1 input channel

model=nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=(3,3), padding=1),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 32, kernel_size=(3,3), padding =1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32,64, kernel_size=(3,3), padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(1536, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64,7)
)

In [219]:
model(faces_train).shape

torch.Size([1030, 7])

Define an Adam optimizer for this model with an initial learning rate of 0.001.

In [ ]:
# for par in model.parameters():
#     print(par.shape)

In [222]:
optimizer= Adam(model.parameters(), 0.001)

Defie a training loop to fit your model faces_train and y_train. Train for 20 epochs in batches of size 32. Every other epoch report the accuracy of the model on the test set.

In [ ]:
model.train()
n_epochs=100
N = faces_train.shape[0]  # total number of observations in training data
batch_size=32

for epoch in range(n_epochs):
  model.train()
  # Shuffle the indices
  indices = torch.randperm(N)

  # Create mini-batches
  for i in range(0, N, batch_size):
    batch_indices = indices[i:i+batch_size]
    batch_X = faces_train[batch_indices]
    batch_y = target_train[batch_indices]

    optimizer.zero_grad()
    predictions = model(batch_X)

    CELoss = torch.nn.CrossEntropyLoss()(predictions, batch_y)

    CELoss.backward()
    optimizer.step()

  if epoch%10==0:
    print(f"epoch: {epoch}, loss: {CELoss.item()}")

epoch: 0, loss: 1.7769851684570312
epoch: 10, loss: 0.19902397692203522
epoch: 20, loss: 0.2995610237121582
epoch: 30, loss: 0.005390379577875137
epoch: 40, loss: 0.045512855052948
epoch: 50, loss: 0.4724731147289276
epoch: 60, loss: 0.6835537552833557
epoch: 70, loss: 0.3021252751350403
epoch: 80, loss: 0.0
epoch: 90, loss: 0.4300449788570404


In [224]:
model.eval()
with torch.no_grad():
    y_pred= model(faces_test)  # -- generate predictions for the test set.
    # accuracy= ((y_pred == target_test).sum / target_test.shape[0]) * 100
accuracy = (y_pred.argmax(dim=1) == target_test).float().mean().item()
# print(f"epoch: {epoch}, accuracy: {accuracy}")
print(f"accuracy: {accuracy}")

accuracy: 0.9534883499145508


Transfer your final accuracy to the file "heomwork21gradescope.py".